Лабораторная работа

Задача: Реализовать модель, которая обучалась бы на финансовых данных (свечи) и предугадывала бы цвет следующих 5 свечей

В своей реализации я использовал данные о валютной паре евро-доллар с 1 апреля 2019 года до 15 апреля 2020 года с интервалом свечи 1 минута

Описание модели:

Используется модель рандомного леса для регрессии с количеством деревьев равным 35.
В качестве зависимых параметров я использовал данные предыдущих относительно текущего момента 100 дней

Параметры:
    1. Разница между ценой открытия и закрытия, выраженная в процентах
    2. Цвет свечи (1 - цена закрытия больше цены открытия, иначе - 0)
    3. Объем
    
Результаты: При обучающей выборке в 50К элементов и при тестовой выборке в 2К элементов удалось добиться точности порядка 60%. При увеличении тестовой выборки до 200К точность стремится к 50%

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("EURUSD_190401_200415.csv", delimiter=',')
data.describe()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
count,3.999610e+05,399961.000000,399961.000000,399961.000000,399961.000000,399961.000000,399961.000000
mean,2.019343e+07,118408.280307,1.110550,1.110646,1.110354,1.110547,1143.022105
std,4.228912e+03,70264.317530,0.013414,0.013401,0.013421,0.013414,1074.164526
min,2.019040e+07,0.000000,1.063800,1.064020,1.063300,1.063700,1.000000
25%,2.019070e+07,55500.000000,1.102290,1.102350,1.102100,1.102280,439.000000
50%,2.019101e+07,120300.000000,1.111440,1.111550,1.111200,1.111430,858.000000
75%,2.020011e+07,181000.000000,1.120320,1.120410,1.120100,1.120320,1506.000000
max,2.020042e+07,235900.000000,1.148360,1.149610,1.147950,1.148260,21508.000000


In [4]:
from sklearn import *

import numpy as np
import math


tmp = data.copy()
tmp = tmp.drop(['<DATE>','<TIME>','<HIGH>','<LOW>'], axis=1)
tmp.rename(columns={'<CLOSE>':'close'}, inplace=True)
tmp.rename(columns={'<OPEN>':'open'}, inplace=True)
tmp.rename(columns={'<VOL>':'vol'}, inplace=True)
tmp['color'] = tmp['close'] >= tmp['open']

prevCandles = 100
predCandles = 3

trainSet = 50000
testSet = 10000

parameters = []

print(len(tmp))

predictColumns = ['color']
for i in range(prevCandles):
    columnName = 'diff' + str(i + 1)
    tmp[columnName] = (tmp['close'].shift(i + 1) - tmp['open'].shift(i + 1)) / tmp['open'].shift(i + 1) * 100
    parameters = parameters + [columnName]
    tmp['color' + str(i+1)] = tmp['color'].shift(i+1)
    parameters = parameters + ['color' + str(i + 1)]
    tmp.loc[abs(tmp[columnName]) < 0.0005, columnName] = 0.0005
    
for i in range(predCandles):
    columnName = 'vol' + str(i + 1)
    tmp[columnName] = tmp['vol'].shift(i + 1)
    parameters = parameters + [columnName]
for i in range(predCandles-1):
    tmp['future_color' + str(i + 1)] = tmp['color'].shift(-i - 1)
    predictColumns = predictColumns + ['future_color' + str(i + 1)]

trainX = np.asarray(tmp.loc[prevCandles:trainSet, parameters])
trainY = np.asarray(tmp.loc[prevCandles:trainSet, predictColumns])

#model = neighbors.KNeighborsRegressor(n_neighbors = 75)
model = ensemble.RandomForestRegressor(n_estimators = 35)
#model = linear_model.LogisticRegression()

model.fit(trainX, trainY)
#print('Coefficient: \n', model.coef_)
#print('Intercept: \n', model.intercept_)
print('R² Value: \n', model.score(trainX, trainY))

testX = np.asarray(tmp.loc[trainSet:trainSet + testSet, parameters])
testY = np.asarray(tmp.loc[trainSet:trainSet + testSet, predictColumns])

predLen = len(predictColumns)

predY = model.predict(testX)

pred = [["NaN" if x == 0.5 else x > 0.5 for x in y] for y in predY] if predLen > 1 else ["NaN" if x == 0.5 else x > 0.5 for x in predY]

count = 0
for i, x in enumerate(pred):
    if predLen > 1:
        score = 0
        for j, y in enumerate(x):
            score += -0.1 if y == "NaN" else 1 if y == testY[i][j] else -1
        if score > 0:
            count += 1
        print('score: ', score)
    else:
        print('Predict value: {}. Real value: {}'.format(x, testY[i][0]))
        if x == testY[i][0]:
            count += 1
print('Вероятность угадывания: ', count / testSet * 100)

399961


/home/wowah/anaconda3/lib/python3.7/site-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


R² Value: 
 0.8482098328339456
score:  3
score:  1
score:  -1
score:  -1
score:  -1
score:  -3
score:  -3
score:  -1
score:  -1
score:  -1
score:  -1
score:  1
score:  3
score:  1
score:  3
score:  1
score:  3
score:  1
score:  -1
score:  1
score:  3
score:  -1
score:  -1
score:  1
score:  -1
score:  -1
score:  -1
score:  -1
score:  -1
score:  1
score:  3
score:  -3
score:  -1
score:  3
score:  3
score:  1
score:  1
score:  -1
score:  -1
score:  -1
score:  1
score:  -3
score:  1
score:  -1
score:  -1
score:  -3
score:  -3
score:  -1
score:  3
score:  1
score:  3
score:  -1
score:  -1
score:  -1
score:  -3
score:  1
score:  -1
score:  3
score:  -1
score:  -1
score:  1
score:  1
score:  3
score:  3
score:  3
score:  3
score:  -3
score:  -1
score:  -1
score:  1
score:  3
score:  -1
score:  3
score:  1
score:  -3
score:  -1
score:  1
score:  -1
score:  -1
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  3
score:  -1
score:  1
score:  -1
score:  -1
score:  3
score:  1
score:  1
sc

score:  -1
score:  -1
score:  3
score:  1
score:  1
score:  -1
score:  1
score:  -1
score:  3
score:  1
score:  -1
score:  -1
score:  1
score:  3
score:  -3
score:  -1
score:  -1
score:  -1
score:  1
score:  1
score:  3
score:  1
score:  1
score:  -3
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  -1
score:  -1
score:  -3
score:  3
score:  -1
score:  1
score:  1
score:  3
score:  1
score:  1
score:  1
score:  -3
score:  -1
score:  -1
score:  1
score:  -1
score:  1
score:  1
score:  1
score:  -1
score:  -1
score:  -3
score:  -1
score:  -1
score:  -1
score:  -1
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  3
score:  3
score:  3
score:  1
score:  3
score:  1
score:  1
score:  3
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  3
score:  3
score:  3
score:  1
score:  -1
score:  1
score:  1
score:  3
score:  1
score:  -1
score:  1
score:  -3
score:  1
score:  -3
score:  -1
score:  -3
score:  -1
score:  -1
score:  3
score:  1
score:  1
score:  -1
sc

score:  -3
score:  1
score:  -3
score:  -1
score:  -1
score:  1
score:  -1
score:  1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  -1
score:  -1
score:  1
score:  -1
score:  1
score:  1
score:  -1
score:  -1
score:  -3
score:  3
score:  1
score:  1
score:  1
score:  -1
score:  -1
score:  1
score:  -1
score:  -1
score:  -1
score:  -1
score:  1
score:  1
score:  -1
score:  1
score:  -1
score:  -3
score:  3
score:  3
score:  -1
score:  -1
score:  -1
score:  3
score:  -3
score:  -1
score:  -1
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  3
score:  1
score:  1
score:  3
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  1
score:  3
score:  -1
score:  -1
score:  -1
score:  1
score:  3
score:  1
score:  -1
score:  -1
score:  1
score:  -1
score:  -3
score:  1
score:  -3
score:  -1
score:  -1
score:  -1
score:  -1
score:  3
score:  -1
score:  3
score:  -1
score:  -3
score:  1
score:  -1
score:  -3
score:  1
score:  -1
score:  -1
score:  1


score:  -1
score:  3
score:  1
score:  -1
score:  1
score:  -1
score:  -3
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  -3
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  3
score:  3
score:  -1
score:  -1
score:  -1
score:  1
score:  1
score:  1
score:  -1
score:  -1
score:  -1
score:  3
score:  -1
score:  -3
score:  1
score:  1
score:  1
score:  1
score:  3
score:  3
score:  -1
score:  -1
score:  -3
score:  -1
score:  -1
score:  -1
score:  1
score:  -3
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  3
score:  -3
score:  -1
score:  -1
score:  1
score:  1
score:  3
score:  3
score:  1
score:  1
score:  -1
score:  -1
score:  -1
score:  -3
score:  1
score:  -1
score:  1
score:  -1
score:  -1
score:  1
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  1
score:  1
score:  1
score:  3
score:  3
score:  -3
score:  -1
score:  1
score:  -3
score:  3
score:  1
score:  -1
score:  -1
score:  -1
score:  1
score:  -1
score:  

score:  3
score:  -1
score:  -1
score:  3
score:  -1
score:  -1
score:  3
score:  -1
score:  1
score:  3
score:  -1
score:  -1
score:  1
score:  -1
score:  -1
score:  -1
score:  3
score:  -1
score:  -1
score:  1
score:  -1
score:  -1
score:  3
score:  1
score:  1
score:  -1
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  -3
score:  1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  -3
score:  3
score:  -1
score:  -1
score:  -1
score:  -1
score:  -1
score:  -1
score:  -1
score:  -1
score:  -1
score:  -1
score:  -3
score:  -1
score:  1
score:  -1
score:  -1
score:  -3
score:  -1
score:  -1
score:  1
score:  1
score:  -1
score:  3
score:  3
score:  3
score:  3
score:  1
score:  -3
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  -3
score:  1
score:  -1
score:  3
score:  1
score:  1
sc

score:  1
score:  -1
score:  -1
score:  -1
score:  -1
score:  -3
score:  -1
score:  1
score:  -1
score:  3
score:  -1
score:  -1
score:  1
score:  -1
score:  3
score:  3
score:  -1
score:  -3
score:  -3
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  3
score:  -1
score:  -1
score:  -1
score:  -1
score:  -3
score:  1
score:  -1
score:  -3
score:  -1
score:  -1
score:  -1
score:  3
score:  -1
score:  -3
score:  -3
score:  1
score:  3
score:  -3
score:  1
score:  1
score:  3
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  -3
score:  -1
score:  1
score:  -3
score:  1
score:  -1
score:  1
score:  3
score:  -3
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  -1
score:  3
score:  -3
score:  -1
score:  -1
score:  1
score:  1
score:  -1
score:  -1
score:  -1
score:  3
score:  1
score:  1
score:  3
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  -3
score:  -3
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  1
sc

score:  -3
score:  1
score:  3
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  -1
score:  1
score:  1
score:  -3
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  -3
score:  1
score:  1
score:  1
score:  -1
score:  3
score:  -1
score:  1
score:  -3
score:  -1
score:  -1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  -1
score:  3
score:  -3
score:  1
score:  -1
score:  1
score:  -1
score:  -3
score:  -3
score:  -3
score:  1
score:  1
score:  3
score:  -1
score:  3
score:  3
score:  1
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  1
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  1
score:  3
score:  -1
score:  1
score:  1
score:  1
score:  1
score:  3
score:  -1
score:  -1
score:  3
score:  1
score:  -1
score:  -1
score:  1
score:  -3
score:  3
score:  -3
score:  1
score:  1
score:  -1
score:  -1
score:  1
score:  1
score:  -1
score:  -1
score:  -1
score:  -1
score:  1
score:  1
scor

score:  -1
score:  -1
score:  -1
score:  3
score:  -1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  -1
score:  1
score:  1
score:  -1
score:  1
score:  1
score:  1
score:  -1
score:  -1
score:  1
score:  1
score:  -1
score:  1
score:  -1
score:  -3
score:  -1
score:  1
score:  -1
score:  3
score:  1
score:  3
score:  3
score:  3
score:  1
score:  1
score:  3
score:  1
score:  -1
score:  1
score:  -1
score:  -3
score:  1
score:  -3
score:  -3
score:  3
score:  3
score:  1
score:  1
score:  3
score:  -1
score:  -1
score:  1
score:  1
score:  1
score:  -1
score:  -1
score:  1
score:  -1
score:  1
score:  1
score:  -1
score:  3
score:  3
score:  -1
score:  1
score:  1
score:  1
score:  -1
score:  -3
score:  -3
score:  1
score:  -3
score:  1
score:  1
score:  3
score:  3
score:  1
score:  3
score:  1
score:  1
score:  -1
score:  -1
score:  1
score:  3
score:  1
score:  3
score:  -1
score:  1
score:  3
score:  1
score:  1
score:  -1
score:  -1
score:  1
score:  1
score:  3
scor